In [2]:
from subtitle_process import fetch_subtitle
import json
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import getpass
import os
import nest_asyncio
import asyncio
import re
from typing import Any, Dict

nest_asyncio.apply()

In [3]:
#从页面所有BV号构建url
urls = []
with open('url.json', 'r') as file:
    urls = json.load(file)

urls = ['https://'+item+'/' for item in urls]

In [4]:
#将前25个视频的字幕加入subtitles
subtitles = []
for url in urls[:5]:
    sub = fetch_subtitle(url)
    subtitles.append(sub)

len(subtitles)

Extracted segment: 13S411A73f
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/19058691521601443768adce9ddfec737839346a06b339dd1f2a?auth_key=1720065253-da591fc3f44f43e395e7daccd0e64f06-0-ee1b0b84ea7e5cb75821432f5bfdce5b
Extracted segment: 1Jf421Q7dF
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/12057938431597546258b9bb9856015fcb212780f97e227e686e?auth_key=1720065262-51a7cf37e2374a7e81cdd5fa0f30c20f-0-27a49d9efb07bf3f55e37c58ce72b2ab
Extracted segment: 1A1421k7JD
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/15559570961597535412b584fccc48b0e4d25ed9edcd9f4b72aa?auth_key=1720065267-7820bebe46de45f798c32470d0858603-0-5e95beae9578c2d161eb227e24d82cb3
Extracted segment: 17z421z7fx
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/135595662015965277167223118aa

5

In [5]:
len(subtitles)

5

In [6]:
template_sub = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一些相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

### 相似题二：
（构造的相似题目二）

### 相似题二讲解：
（构造的相似题目二的讲解）

视频字幕：
{subtitle}

注意：对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话。
"""

In [7]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [8]:
'''
输入一：一个包含template中所有变量以及对应值的字典
输入二：一个给gpt的prompt template
输入三：回答token数量限制, 默认1000
输出：gpt的回答
'''
async def inference(params: Dict[str, any], template: str, max_tokens=1000) -> str:
    
    output = ""
    llm = ChatOpenAI(
        model='gpt-4o',
        temperature=0,
        max_tokens=max_tokens,
        max_retries=2,
        timeout=20,
    )
    print("executing")

    prompt = PromptTemplate.from_template(template=template)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    async for chunk in chain.astream(params, version="v2"):
        output += chunk
        print(chunk, end='', flush=True)

    return output

In [9]:
a = await inference({"subtitle" : subtitles[0]}, template_sub)
a

executing
### 原题：
在一次羽毛球赛中，八个队进行循环赛，需要比赛几场？

### 讲解：
首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有八个队，所以你需要和其他七个队比赛，即七场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用一个简单的数学方法来计算：每个队都需要和其他七个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：

总场数 = 7 + 6 + 5 + 4 + 3 + 2 + 1

我们可以用凑整法来快速计算这个和：

7 + 6 = 13
13 + 5 = 18
18 + 4 = 22
22 + 3 = 25
25 + 2 = 27
27 + 1 = 28

所以，总共需要进行28场比赛。

### 相似题一：
在一次乒乓球赛中，六个队进行循环赛，需要比赛几场？

### 相似题一讲解：
首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有六个队，所以你需要和其他五个队比赛，即五场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用一个简单的数学方法来计算：每个队都需要和其他五个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：

总场数 = 5 + 4 + 3 + 2 + 1

我们可以用凑整法来快速计算这个和：

5 + 4 = 9
9 + 3 = 12
12 + 2 = 14
14 + 1 = 15

所以，总共需要进行15场比赛。

### 相似题二：
在一次篮球赛中，十个队进行循环赛，需要比赛几场？

### 相似题二讲解：
首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有十个队，所以你需要和其他九个队比赛，即九场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用一个简单的数学方法来计算：每个队都需要和其他九个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：

总场数 = 9 + 8 + 7 + 6 + 5 + 4 + 3 + 2 + 1

我们可以用凑整法来快速计算这个和：

9 + 8 = 17
17 + 7 = 24
24 + 6 = 30
30 + 5 = 

'### 原题：\n在一次羽毛球赛中，八个队进行循环赛，需要比赛几场？\n\n### 讲解：\n首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有八个队，所以你需要和其他七个队比赛，即七场。接下来，我们需要计算所有队伍之间的比赛总场数。\n\n我们可以用一个简单的数学方法来计算：每个队都需要和其他七个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：\n\n总场数 = 7 + 6 + 5 + 4 + 3 + 2 + 1\n\n我们可以用凑整法来快速计算这个和：\n\n7 + 6 = 13\n13 + 5 = 18\n18 + 4 = 22\n22 + 3 = 25\n25 + 2 = 27\n27 + 1 = 28\n\n所以，总共需要进行28场比赛。\n\n### 相似题一：\n在一次乒乓球赛中，六个队进行循环赛，需要比赛几场？\n\n### 相似题一讲解：\n首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有六个队，所以你需要和其他五个队比赛，即五场。接下来，我们需要计算所有队伍之间的比赛总场数。\n\n我们可以用一个简单的数学方法来计算：每个队都需要和其他五个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：\n\n总场数 = 5 + 4 + 3 + 2 + 1\n\n我们可以用凑整法来快速计算这个和：\n\n5 + 4 = 9\n9 + 3 = 12\n12 + 2 = 14\n14 + 1 = 15\n\n所以，总共需要进行15场比赛。\n\n### 相似题二：\n在一次篮球赛中，十个队进行循环赛，需要比赛几场？\n\n### 相似题二讲解：\n首先，我们考虑如果你是其中的一个队，你需要和其他几个队比赛。因为一共有十个队，所以你需要和其他九个队比赛，即九场。接下来，我们需要计算所有队伍之间的比赛总场数。\n\n我们可以用一个简单的数学方法来计算：每个队都需要和其他九个队比赛，但这样每场比赛会被计算两次（因为A对B和B对A是同一场比赛）。所以我们可以用以下公式来计算总场数：\n\n总场数 = 9 + 8 + 7 + 6 + 5 + 4 + 3 + 2 + 1\n\n我们可以用凑整法来快速计算这个和：\n\n9

In [10]:

def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)\n\n### 相似题二：\n(.+?)\n\n### 相似题二讲解：\n(.+?)\n", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
                "similar_question_2": match[4].strip(),
                "similar_explanation_2": match[5].strip(),
            })

    with open('questions_explanation.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)

In [11]:
responses = []
for subtitle in subtitles:
    response = await inference({"subtitle": subtitle}, template_sub, 1000)
    responses.append(response)

extract_question_explanations(responses)

executing
### 原题：
在一次羽毛球赛中，八个队进行循环赛，需要比赛几场？

### 讲解：
在这次羽毛球赛中，有八个队进行循环赛。我们可以先考虑一个队需要和其他几个队比赛。假设你是其中的一个队，你需要和其他七个队进行比赛，所以你需要比赛七场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用组合数学中的组合公式来计算，即C(8, 2)，表示从8个队中选出2个队进行比赛的组合数。公式为：
\[ C(n, 2) = \frac{n(n-1)}{2} \]
代入n=8，得到：
\[ C(8, 2) = \frac{8 \times 7}{2} = 28 \]
所以，总共需要进行28场比赛。

### 相似题一：
在一次篮球赛中，十个队进行循环赛，需要比赛几场？

### 相似题一讲解：
在这次篮球赛中，有十个队进行循环赛。我们可以先考虑一个队需要和其他几个队比赛。假设你是其中的一个队，你需要和其他九个队进行比赛，所以你需要比赛九场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用组合数学中的组合公式来计算，即C(10, 2)，表示从10个队中选出2个队进行比赛的组合数。公式为：
\[ C(n, 2) = \frac{n(n-1)}{2} \]
代入n=10，得到：
\[ C(10, 2) = \frac{10 \times 9}{2} = 45 \]
所以，总共需要进行45场比赛。

### 相似题二：
在一次乒乓球赛中，六个队进行循环赛，需要比赛几场？

### 相似题二讲解：
在这次乒乓球赛中，有六个队进行循环赛。我们可以先考虑一个队需要和其他几个队比赛。假设你是其中的一个队，你需要和其他五个队进行比赛，所以你需要比赛五场。接下来，我们需要计算所有队伍之间的比赛总场数。

我们可以用组合数学中的组合公式来计算，即C(6, 2)，表示从6个队中选出2个队进行比赛的组合数。公式为：
\[ C(n, 2) = \frac{n(n-1)}{2} \]
代入n=6，得到：
\[ C(6, 2) = \frac{6 \times 5}{2} = 15 \]
所以，总共需要进行15场比赛。executing
### 原题：
在一次足球比赛中，四个队进行循环赛，需要比赛多少场？（两个队之间比赛一次算一场）

### 讲解：
首先，我们解释一下什么叫

In [12]:
subtitles[0]

'接下来看我们的练习五的第一题一在一次羽毛球赛中八个队进行循环赛需要比赛几场那根据我们刚才老师给大家说的那请问八个队如果你代表的是其中的一个对你需要比赛几场大家先考虑这个问题然后答出来把答案答出来来这八个队里边如果你是其中的一个队你需要和几个队比赛比几场考虑一下糖宝都对了嗯机场呀对七场你需要和其他七个队进行比赛所以是七就是我们开始的数字干嘛呀加嘛一直往后加加到一所以是7+6加5+4加3+2加一算一下你就知道等于多少了这个可以怎么算给我发一个来这个可以怎么算你可以起巧算的吧7+346+40再加五呢十五十五再加上一个二加上一个一等于多少呀哎28场来听懂的给老师打个一谢谢听懂的给老师打个一等差数列是也对但是它比较短一点我们可以用我们这个凑整法啊可以去快速的计算的O'

In [3]:
from functions import inference
from templates import template_sub
await inference({"subtitle": "接下来看我们的练习五的第一题一在一次羽毛球赛中八个队进行循环赛需要比赛几场那根据我们刚才老师给大家说的那请问八个队如果你代表的是其中的一个对你需要比赛几场大家先考虑这个问题然后答出来把答案答出来来这八个队里边如果你是其中的一个队你需要和几个队比赛比几场考虑一下糖宝都对了嗯机场呀对七场你需要和其他七个队进行比赛所以是七就是我们开始的数字干嘛呀加嘛一直往后加加到一所以是7+6加5+4加3+2加一算一下你就知道等于多少了这个可以"}, template_sub)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [1]:
from problem_from_subtitles import run
await run()

Extracted segment: 11y421h7FQ
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/1550499769143512359518d70356f5d95d6b6450832b1d176906?auth_key=1720077996-cda547f5d4b3455280c811b7b51a87de-0-7e29c1622d5f35d697f9492f3c2b7664
Extracted segment: 1hC41147ca
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/19504979461435123675d8442fd1d7219219c0796c1b3502b071?auth_key=1720078000-13763c4666c1470bb51bf50fa923b380-0-0f3758e19d60e7d364e8e6516214e2bf
Extracted segment: 1gN4y1W7cX
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/88054547813888580749a0a41aef5ce5c77869108c1f35cc565?auth_key=1720078005-f4bdab0a373445828b0579bd0078eddf-0-3016192c0586dd85a431331aa1e406b5
executing
### 原题：
用红、黄、蓝、紫四种颜色的笔涂下面的四个圆圈，而且四个圆圈的颜色都不一样，共有几种涂法？

### 讲解：
这道题目要求我们用红、黄、蓝、紫四种颜色的笔涂四个圆圈，并且每个圆圈的颜色都不一样。我们可以通过排列组合的方法来解决这个问题。

首先，我们考虑第一个圆圈的颜色。因为有四种颜色可以选择，所以

In [6]:
from templates import template_note, template_expl, template_orig, template_sub, ref_form, ref
from functions import inference
import json

In [4]:
# 通过json list，返回一个format后的问题，讲解string
def formatter(data):
    s = ""
    for gp in data:
        s += "###题目\n" + gp['original_question'] + "\n"
        s += "###讲解\n" + gp['original_explanation'] + '\n\n'
    return s

with open("questions_explanation.json", 'r') as f:
    data = json.load(f)
few_shots = formatter(data)
few_shots

'###题目\n用红、黄、蓝、紫四种颜色的笔涂下面的四个圆圈，而且四个圆圈的颜色都不一样，共有几种涂法？\n###讲解\n这道题目要求我们用红、黄、蓝、紫四种颜色的笔涂四个圆圈，并且每个圆圈的颜色都不一样。我们可以通过排列组合的方法来解决这个问题。\n\n首先，我们考虑第一个圆圈的颜色。因为有四种颜色可以选择，所以第一个圆圈有4种选择。\n\n接下来，考虑第二个圆圈的颜色。由于第一个圆圈已经使用了一种颜色，剩下的颜色有3种选择。\n\n然后，考虑第三个圆圈的颜色。此时，前两个圆圈已经使用了两种颜色，剩下的颜色有2种选择。\n\n最后，第四个圆圈的颜色。前三个圆圈已经使用了三种颜色，剩下的颜色只有1种选择。\n\n因此，总共有 \\(4 \\times 3 \\times 2 \\times 1 = 24\\) 种不同的涂法。\n\n###题目\n小琪有三件不同颜色的上衣和四件不同颜色的裙子，问她共有多少种不同的穿搭？\n###讲解\n这道题目考察的是排列组合的基本原理。我们有三件不同颜色的上衣和四件不同颜色的裙子。每一件上衣都可以和每一件裙子进行搭配。我们可以通过以下步骤来计算总的搭配数：\n\n1. 首先，我们将三件上衣分别标记为1、2、3。\n2. 然后，将四件裙子分别标记为A、B、C、D。\n3. 接下来，我们列举出每一件上衣与每一件裙子的所有搭配情况。\n\n当上衣是1时，它可以搭配的裙子有：1A、1B、1C、1D，共4种搭配。\n当上衣是2时，它可以搭配的裙子有：2A、2B、2C、2D，共4种搭配。\n当上衣是3时，它可以搭配的裙子有：3A、3B、3C、3D，共4种搭配。\n\n因此，每一件上衣都有4种不同的搭配方式，总共有3件上衣，所以总的搭配数为：\n3 × 4 = 12种。\n\n###题目\n用两个四和十个零组成一个12位数，要求每一集的末尾零都不读。\n###讲解\n同学们，首先我们要明确题目的要求：用两个四和十个零组成一个12位数，并且每一集的末尾零都不读。我们可以这样来做：\n\n1. 先把两个四和十个零写出来：440000000000。\n2. 按照题目的要求，每一集的末尾零都不读，所以我们要把零放在每一集的末尾。\n3. 这样我们可以得到：四百四十亿。\n\n注意：每一集的末尾零都不读，所以我们只读一个零。\n\n'

In [9]:
'''
函数步骤：
1.生成板书
2.将板书，以及之前生成的模板讲解放入生成讲解的 prompt
3.生成讲解
'''
question = "A,B,C三个人去游乐园玩，三个人在藏宝五种一共发现了5件宝物，这三个人找到的宝物数量可能有多少种情况？"
notes = await inference({"question": question, "ref_form": ref_form, "ref": ref}, template_note)

explanation = await inference({"question": question, "ref": ref, "notes": notes, "model_expl": few_shots}, template_expl)
original_response = await inference({"question": question, "ref_form": ref_form, "ref": ref}, template_orig)

executing
知识点讲解:本题考查的知识点是组合数学中的分配问题，解题关键点是计算不同的分配方式。

###板书一:
设A、B、C三个人分别找到的宝物数量为$a, b, c$，则有
$a + b + c = 5$

###板书二:
根据分配问题的公式，$a + b + c = n$的非负整数解的个数为$\binom{n+k-1}{k-1}$，其中$n$是总数，$k$是分配的组数。
$\therefore \binom{5+3-1}{3-1} = \binom{7}{2} = 21$

故答案是21种情况。executing
###讲解一:
同学们，这道题目考查的是组合数学中的分配问题。我们需要计算出A、B、C三个人在藏宝五种中找到5件宝物的不同分配方式。首先，我们设A、B、C三个人分别找到的宝物数量为$a, b, c$，那么我们可以写出一个等式：
\[ a + b + c = 5 \]
这个等式表示三个人找到的宝物数量之和为5。

###讲解二:
接下来，我们需要计算这个等式的非负整数解的个数。根据组合数学中的分配问题公式，$a + b + c = n$的非负整数解的个数为：
\[ \binom{n+k-1}{k-1} \]
其中，$n$是总数，$k$是分配的组数。在这道题目中，$n = 5$，$k = 3$，所以我们代入公式：
\[ \binom{5+3-1}{3-1} = \binom{7}{2} \]
接着，我们计算组合数：
\[ \binom{7}{2} = \frac{7 \times 6}{2 \times 1} = 21 \]
因此，三个人找到的宝物数量有21种不同的分配情况。答案是21种情况。executing


/var/folders/d1/0cl9ndvx4q38ykjb5ycmp4zr0000gn/T/ipykernel_54216/513395221.py:10: RuntimeWarning: coroutine 'inference' was never awaited
  explanation = await inference({"question": question, "ref": ref, "notes": notes, "model_expl": few_shots}, template_expl)


知识点讲解:本题考查的知识点是组合与分配问题，解题关键点是利用分配方法计算不同的分配情况。

###板书一:
解：设A、B、C三个人分别找到的宝物数量为$a, b, c$，则有
$a + b + c = 5$

###讲解一:
首先，我们设A、B、C三个人分别找到的宝物数量为$a, b, c$，根据题意，三个人找到的宝物总数为5，因此有$a + b + c = 5$。

###板书二:
$\because$ $a, b, c$均为非负整数
$\therefore$ 该问题转化为在5个宝物中插入2个隔板的问题
$\binom{5+2}{2} = \binom{7}{2} = 21$

###讲解二:
因为$a, b, c$均为非负整数，所以我们可以将问题转化为在5个宝物中插入2个隔板的问题。根据组合数公式，插入2个隔板的方法数为$\binom{5+2}{2} = \binom{7}{2} = 21$。

###板书三:
故答案是：21种情况

###讲解三:
通过计算，我们得出共有21种不同的分配情况。故答案是21种情况。

/var/folders/d1/0cl9ndvx4q38ykjb5ycmp4zr0000gn/T/ipykernel_54216/513395221.py:11: RuntimeWarning: coroutine 'inference' was never awaited
  original_response = await inference({"question": question, "ref_form": ref_form, "ref": ref}, template_orig)


In [10]:
notes

'知识点讲解:本题考查的知识点是组合数学中的分配问题，解题关键点是计算不同的分配方式。\n\n###板书一:\n设A、B、C三个人分别找到的宝物数量为$a, b, c$，则有\n$a + b + c = 5$\n\n###板书二:\n根据分配问题的公式，$a + b + c = n$的非负整数解的个数为$\\binom{n+k-1}{k-1}$，其中$n$是总数，$k$是分配的组数。\n$\\therefore \\binom{5+3-1}{3-1} = \\binom{7}{2} = 21$\n\n故答案是21种情况。'

In [11]:
explanation

'###讲解一:\n同学们，这道题目考查的是组合数学中的分配问题。我们需要计算出A、B、C三个人在藏宝五种中找到5件宝物的不同分配方式。首先，我们设A、B、C三个人分别找到的宝物数量为$a, b, c$，那么我们可以写出一个等式：\n\\[ a + b + c = 5 \\]\n这个等式表示三个人找到的宝物数量之和为5。\n\n###讲解二:\n接下来，我们需要计算这个等式的非负整数解的个数。根据组合数学中的分配问题公式，$a + b + c = n$的非负整数解的个数为：\n\\[ \\binom{n+k-1}{k-1} \\]\n其中，$n$是总数，$k$是分配的组数。在这道题目中，$n = 5$，$k = 3$，所以我们代入公式：\n\\[ \\binom{5+3-1}{3-1} = \\binom{7}{2} \\]\n接着，我们计算组合数：\n\\[ \\binom{7}{2} = \\frac{7 \\times 6}{2 \\times 1} = 21 \\]\n因此，三个人找到的宝物数量有21种不同的分配情况。答案是21种情况。'

In [12]:
original_response

'知识点讲解:本题考查的知识点是组合与分配问题，解题关键点是利用分配方法计算不同的分配情况。\n\n###板书一:\n解：设A、B、C三个人分别找到的宝物数量为$a, b, c$，则有\n$a + b + c = 5$\n\n###讲解一:\n首先，我们设A、B、C三个人分别找到的宝物数量为$a, b, c$，根据题意，三个人找到的宝物总数为5，因此有$a + b + c = 5$。\n\n###板书二:\n$\\because$ $a, b, c$均为非负整数\n$\\therefore$ 该问题转化为在5个宝物中插入2个隔板的问题\n$\\binom{5+2}{2} = \\binom{7}{2} = 21$\n\n###讲解二:\n因为$a, b, c$均为非负整数，所以我们可以将问题转化为在5个宝物中插入2个隔板的问题。根据组合数公式，插入2个隔板的方法数为$\\binom{5+2}{2} = \\binom{7}{2} = 21$。\n\n###板书三:\n故答案是：21种情况\n\n###讲解三:\n通过计算，我们得出共有21种不同的分配情况。故答案是21种情况。'